# Installing packages

In [ ]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install "torch==2.4.0" "xformers==0.0.27.post2" triton torchvision torchaudio "trl==0.14.0"
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"
!pip uninstall -y trl
!pip install git+https://github.com/huggingface/trl.git@e95f9fb74a3c3647b86f251b7e230ec51c64b72b

# Import Libs

In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn.functional as F
from unsloth import FastLanguageModel

import os
import ast
import string
import random
from tqdm.auto import tqdm
from IPython.display import clear_output

os.environ["WANDB_DISABLED"] = "true"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# Downloading model

In [ ]:
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"

# >>> Download LLM Model
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length = 8192,
    dtype = None,
    load_in_4bit = True,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)
FastLanguageModel.for_inference(model)
print("Model is ready to inference!")

# Load & process dataset

In [ ]:
df_train = pd.read_csv("/kaggle/input/fpt-ai-residency-batch-6-entry-test/b6_train_data.csv")
df_train.head()

In [ ]:
df_test = pd.read_csv("/kaggle/input/fpt-ai-residency-batch-6-entry-test/b6_test_data.csv")
df_test.head()

In [ ]:
df_train.describe()

In [ ]:
df_train = df_train.dropna()
df_train = df_train.drop_duplicates()
df_train.describe()

In [ ]:
df_test.describe()

In [ ]:
prompt_tempalte = """You are a programming expert and will answer multiple-choice questions about code.  
Read the following code snippet carefully and select the **best** answer.  

### Response Format:
- Reply with **only** the letter of the correct choice (A, B, C, or D).  
- Do **not** provide explanations.  

### Question:
{}

### Choices:
{}

### Response:
"""

In [ ]:
index2choice = {i: letter for i, letter in enumerate(string.ascii_uppercase)}
choice2index = {letter: i for i, letter in enumerate(string.ascii_uppercase)}

def choices2str(choices):
    choices_lst = ast.literal_eval(choices)
    result = ""
    for i in range(0, len(choices_lst)):
        result = result + index2choice[i] + ". " + str(choices_lst[i]) + "\n\n"
    return result.strip()

def apply_template(question, choices):
    return prompt_tempalte.format(question, choices2str(choices))

In [ ]:
print(apply_template(df_train['question'].iloc[0], df_train['choices'].iloc[0]))

In [ ]:
df_train['template'] = df_train.apply(lambda x: apply_template(x['question'], x['choices']), axis=1)
df_train.head()

In [ ]:
df_test['template'] = df_test.apply(lambda x: apply_template(x['question'], x['choices']), axis=1)
df_test.head()

# Response Parser

In [ ]:
def response_parser(response):
    if len(response) == 1 and response.upper() in list(choice2index.keys()):
        return response

    response = response.translate(str.maketrans('', '', string.punctuation))
    response_split = response.split()
    for re in response_split:
        if re in list(choice2index.keys()):
            return re
    return random.choice(list(choice2index.keys()))

# Inference

In [ ]:
def generate(prompt:str, max_new_tokens=1):
        inputs = tokenizer(
            [prompt], return_tensors="pt"
        ).to("cuda")
        input_length = inputs.input_ids.shape[1]  # Lưu độ dài ban đầu
        
        output_ids = model.generate(**inputs, max_new_tokens=max_new_tokens, use_cache=True)
        new_tokens = output_ids[0, input_length:]
        response = tokenizer.decode(new_tokens, skip_special_tokens=True)

        return response.strip()

In [ ]:
generate("Mày ngu")

In [ ]:
data_dict = {
    "task_id": [],
    "answer": []
}

In [ ]:
for i in tqdm(range(0, df_test.shape[0]), desc="Inference on Test set"):
    task_id = df_test['task_id'].iloc[i]
    prompt = df_test['template'].iloc[i]
    data_dict['task_id'].append(task_id)
    response = generate(prompt, 2)
    answer = response_parser(response)
    data_dict['answer'].append(answer)

# Submission

In [ ]:
submission = pd.DataFrame(data_dict)
submission.head()

In [ ]:
submission.describe()

In [ ]:
submission.to_csv("submission.csv", index=False)